In [1]:
from cryptography import x509
from cryptography.hazmat.primitives import serialization
from cryptography.hazmat.primitives.asymmetric import rsa
from cryptography.hazmat.primitives import hashes
import datetime

In [26]:
def create_ca(common_name: str):
    private_key_ca = rsa.generate_private_key(public_exponent=65537, key_size=2048)
    
    x509_common_name = x509.NameAttribute(x509.NameOID.COMMON_NAME, common_name)
    
    subject_name_ca = x509.Name([x509_common_name])
    issuer_ca = x509.Name([x509_common_name])
    
    cert_ca = x509.CertificateBuilder() \
        .subject_name(subject_name_ca) \
        .issuer_name(issuer_ca) \
        .public_key(private_key_ca.public_key()) \
        .serial_number(x509.random_serial_number()) \
        .not_valid_before(datetime.datetime.utcnow()) \
        .not_valid_after(datetime.datetime.utcnow() + datetime.timedelta(days=3650)) \
        .sign(private_key_ca, hashes.SHA256())

    return cert_ca, private_key_ca


def create_csr(common_name: str, email: str=None):
    private_key_csr = rsa.generate_private_key(public_exponent=65537, key_size=2048)

    csr_builder = x509.CertificateSigningRequestBuilder()
    
    subject_name_csr = x509.Name([x509.NameAttribute(x509.NameOID.COMMON_NAME, common_name)])
    csr_builder = csr_builder.subject_name(subject_name_csr)

    if email:
        san_email = x509.SubjectAlternativeName([x509.RFC822Name(email)])
        csr_builder = csr_builder.add_extension(san_email)
    
    csr = csr_builder.sign(private_key_csr, hashes.SHA256())

    return csr, private_key_csr


def issue_certificate(cert_ca, private_key_ca, csr):
    cert_signed = x509.CertificateBuilder() \
        .subject_name(csr.subject) \
        .issuer_name(cert_ca.subject) \
        .public_key(csr.public_key()) \
        .serial_number(x509.random_serial_number()) \
        .not_valid_before(datetime.datetime.utcnow()) \
        .not_valid_after(datetime.datetime.utcnow() + datetime.timedelta(days=365)) \
        .sign(private_key_ca, hashes.SHA256())

    return cert_signed


def save_private_key(private_key, filename):
    with open(filename, "wb") as f:
        f.write(private_key.private_bytes(
            encoding=serialization.Encoding.PEM,
            format=serialization.PrivateFormat.TraditionalOpenSSL,
            encryption_algorithm=serialization.NoEncryption()
        ))


def save_certificate(cert, filename):
    with open(filename, "wb") as f:
        f.write(cert.public_bytes(serialization.Encoding.PEM))    